<a href="https://colab.research.google.com/github/Meghana09G/final-exam-2/blob/main/700761933_final_exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.utils import to_categorical

# Load and clean data
data = pd.read_csv('/root/Data.csv')

data = data[['text', 'sentiment']]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]', '', x))
data['text'] = data['text'].apply(lambda x: x.replace("rt", " "))

# Tokenization
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

# Encode labels
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Encode sentiment labels
labelencoder = LabelEncoder()  # <- this line must be executed first
integer_encoded = labelencoder.fit_transform(data['sentiment'])  # e.g., ['positive', 'negative', 'neutral'] → [2, 0, 1]
y = to_categorical(integer_encoded)  # Convert to one-hot


# Split data
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Build model
embed_dim = 128
lstm_out = 196
model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))  # Assuming 3 sentiment classes
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train
batch_size = 32
model.fit(X_train, Y_train, epochs=1, batch_size=batch_size, verbose=2)

# Evaluate
score, acc = model.evaluate(X_test, Y_test, verbose=2, batch_size=batch_size)
print("Test Loss:", score)
print("Test Accuracy:", acc)

# Save model, tokenizer, and label encoder
model.save("sentiment_lstm_model.h5")

with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

with open('labelencoder.pkl', 'wb') as f:
    pickle.dump(labelencoder, f)

print("Model, tokenizer, and label encoder saved successfully.")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


291/291 - 42s - 143ms/step - accuracy: 0.6426 - loss: 0.8292
144/144 - 3s - 21ms/step - accuracy: 0.6638 - loss: 0.7758


Test Loss: 0.7758232355117798
Test Accuracy: 0.6638270020484924
Model, tokenizer, and label encoder saved successfully.
